In [1]:
# To copy list without reference
import copy

In [2]:
# Check solvability with inversions
def check_solvability(startState, goalState):
    goalInversions = 0
    startInversions = 0
    
    oneDimStart = []
    oneDimGoal = []
    
#     Creating one dimensional array and then adding the inversions together
#   Number of inversions must match
    for i in range(len(startState)):
        for j in range(len(startState[i])):
            oneDimStart.append(startState[i][j])
            oneDimGoal.append(goalState[i][j])   
    
    for i in range(9):
        for j in range(9-i):
            if oneDimGoal[i] < oneDimGoal[i+j]:
                if(oneDimGoal[i] != 0):
                    goalInversions = goalInversions+1
                
            if oneDimStart[i] < oneDimStart[i+j]:
                if(oneDimStart[i] != 0):
                    startInversions = startInversions+1
                    
    if goalInversions%2 != startInversions%2:
        raise Exception("Goal-State chan not be Achieved with the given Input-State")
    
    print("Input state is solvable!")   
    
    
# Checking for different wrong inputs    
def check_validity(state):
#   Check if input dimension is correct        
    if len(state) != 3:
        raise Exception("Not supported input dimension. Must be 3x3 matrix")
    for i in range(len(state)):
        if len(state[i]) != 3:
            raise Exception("Not supported input dimension. Must be 3x3 matrix")

             
#   Check if all input elements is an integer
    for i in range(len(state)):
        for j in range(len(state[i])):
            if type(state[i][j]) != int:
#                 print(i, j, state[i][j])
                raise Exception("Input must be an integer")

# #   Check if only valid Digits (0-8)
    for i in range(len(state)):
        for j in range(len(state[i])):
            if state[i][j] > 8 or state[i][j] < 0:
                raise Exception("All input elements must be in range 0-8")
            
# #    Check if all entries are unique            
    oneDimList = []
    for i in range(len(state)):
        for j in range(len(state[i])):
            oneDimList.append(state[i][j])
    
    for i in range(len(oneDimList)-1):
        if sorted(oneDimList)[i] == sorted(oneDimList)[i+1]:
            raise Exception("All entries must be unique")

    print("Input state is valid!")
    

# Detecting blank (0) tile
def blankTileLocation(state):
    for i in range(len(state)):
        for j in range(len(state[i])):
            if state[i][j] == 0:
                return [i, j]
            
# Function to make a number array from matrix            
def mat2num(state):
    iteration = 0;
    num = 0;
    for i in range(len(state)):
        for j in range(len(state[i])):
            num += (state[i][j] * pow(10, 8-iteration))
            iteration += 1;
            
    return num


# Check if new state is actually new. Comparing numeric form (not matrix) to make it run faster
def checkNewState(newState, stateList):
    output = True
    newStateNum = mat2num(newState)
    for i in range(len(stateList)):
        for j in range(len(stateList[i])):
            if newStateNum == stateList[i][j][STATE_NUM_POS]:
                output = False
            else:
                output = True
    return output
                

#  Action Functions
def actionMoveUp(stateToMove):
    movedState = stateToMove
    [i, j] = blankTileLocation(movedState)
    numMove = movedState[i-1][j]
    movedState[i][j] = int(numMove)
    movedState[i-1][j] = 0
    
    return [movedState, "Up"]



def actionMoveDown(stateToMove):
    movedState = stateToMove
    [i, j] = blankTileLocation(stateToMove)
    numMove = stateToMove[i+1][j]
    movedState[i][j] = numMove
    movedState[i+1][j] = 0
    
    return [movedState, "Down"]

def actionMoveLeft(stateToMove):
    movedState = stateToMove
    [i, j] = blankTileLocation(stateToMove)
    numMove = stateToMove[i][j-1]
    movedState[i][j] = numMove
    movedState[i][j-1] = 0
    
    return [movedState, "Left"]

def actionMoveRight(stateToMove):
    movedState = stateToMove
    [i, j] = blankTileLocation(stateToMove)
    numMove = stateToMove[i][j+1]
    movedState[i][j] = numMove
    movedState[i][j+1] = 0
    
    return [movedState, "Right"]



# Back Tracing from Goal State
def backTracing(solutionPos, stateList):
    backTraceList = [stateList[solutionPos[0]][solutionPos[1]]]
    
    solutionLevel = solutionPos[0]
        
    for i in range(solutionPos[0]):
        backTraceList.append(stateList[backTraceList[i][PARENT_LEVEL_POS]][backTraceList[i][PARENT_ITERATION_POS]])
        
    return backTraceList



# Commands to write text files
def writeNodes(node, textFileObject):
    for i in range(len(node[0])):
        for j in range(len(node[0][i])):
            textFileObject.write(str(node[0][j][i])+ ' ')
    textFileObject.write("\n")
    
def writeInfo(node, textFileObject):
    textFileObject.write("\t" + str(i) + "," + str(node[ITERATION_POS]) + '\t\t\t\t' + str(node[PARENT_LEVEL_POS]) + ',' + str(node[PARENT_ITERATION_POS]))
    textFileObject.write("\n")       
            
        
            

In [5]:
# Give Input and Goal State as 3x3 List
input_state = [[1, 2, 3], [4, 5, 6], [7, 0, 8]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

check_validity(input_state)
check_solvability(input_state, goal_state)

Input state is valid!
Input state is solvable!


In [6]:
# Global Variables Creation
solutionState = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
solutionPos = [0, 0]  #Level and Iteration

# Get Goal setup as decNumber
goalStateNum = mat2num(goal_state)

# CONSTANTS for array positions
STATE_POS = 0
PARENT_LEVEL_POS = 1
PARENT_ITERATION_POS = 2
ZERO_POS = 3
ITERATION_POS = 4
STATE_NUM_POS = 5
ORIGIN_POS = 6


# Global Variables
level = 0
currentState = copy.deepcopy(input_state)
currentLevel = 0
stateList = []
solutionFound = False
parent_level = 0
parent_iteration = 0

# Find first blanTileLocation
[iZero, jZero] = blankTileLocation(copy.deepcopy(currentState))

# Append first level of states
stateList.append([])

# Data strucuted as followd: stateList[Level of new Solutions derived from previous level][Iterations within Level] ...
#                               ...[state(as 2d list), parentLevel, parentIteration, ZeroPosition(i,j), currentLevel, state as decNumber, direction moved last]
stateList[0].append([currentState.copy(), parent_level, parent_iteration, [iZero, jZero], level, mat2num(currentState.copy()), "noDirection"])

level = 0
while solutionFound == False:
    stateList.append([])
    
    for iteration in range(len(stateList[level])):
        [iZero, jZero] = blankTileLocation(copy.deepcopy(stateList[level][iteration][STATE_POS]))
        if iZero > 0 and stateList[level][iteration][ORIGIN_POS] != "Down":
            up = True
            [newState, origin] = actionMoveUp(copy.deepcopy(stateList[level][iteration][STATE_POS]))
            if checkNewState(newState, stateList):
                stateList[level+1].append([newState, level, iteration, blankTileLocation(newState), len(stateList[level+1]), mat2num(newState), origin])
            
            if mat2num(newState) == goalStateNum:
                stateList[level+1].append([newState, level, iteration, blankTileLocation(newState), len(stateList[level+1]), mat2num(newState), origin])
                solutionFound = True
                solutionState = newState
                solutionPos = [level, iteration]
                break
                

        if iZero < 2 and stateList[level][iteration][ORIGIN_POS] != "Up":
            down = True
            [newState, origin] = actionMoveDown(copy.deepcopy(stateList[level][iteration][STATE_POS]))
            if checkNewState(newState, stateList):
                stateList[level+1].append([newState, level, iteration, blankTileLocation(newState), len(stateList[level+1]), mat2num(newState), origin])
            
            if mat2num(newState) == goalStateNum:
                stateList[level+1].append([newState, level, iteration, blankTileLocation(newState), len(stateList[level+1]), mat2num(newState), origin])
                solutionFound = True
                solutionState = newState
                solutionPos = [level, iteration]
                break

        if jZero > 0 and stateList[level][iteration][ORIGIN_POS] != "Right":
            left = True
            [newState, origin] = actionMoveLeft(copy.deepcopy(stateList[level][iteration][STATE_POS]))
            if checkNewState(newState, stateList):
                stateList[level+1].append([newState, level, iteration, blankTileLocation(newState), len(stateList[level+1]), mat2num(newState), origin])
            
            if mat2num(newState) == goalStateNum:
                stateList[level+1].append([newState, level, iteration, blankTileLocation(newState), len(stateList[level+1]), mat2num(newState), origin])
                solutionFound = True
                solutionState = newState
                solutionPos = [level, iteration]
                break

        if jZero < 2 and stateList[level][iteration][ORIGIN_POS] != "Left":
            right = True
            [newState, origin] = actionMoveRight(copy.deepcopy(stateList[level][iteration][STATE_POS]))
            if checkNewState(newState, stateList):
                stateList[level+1].append([newState, level, iteration, blankTileLocation(newState), len(stateList[level+1]), mat2num(newState), origin])
            
            if mat2num(newState) == goalStateNum:
                stateList[level+1].append([newState, level, iteration, blankTileLocation(newState), len(stateList[level+1]), mat2num(newState), origin])
                solutionFound = True
                solutionState = newState
                solutionPos = [level+1, len(stateList[level+1])-1]
                break
    level = level + 1
    if solutionFound == True:
        break
        

backTraceList = backTracing(solutionPos, stateList)
    

for i in range(solutionPos[0]+1):
    for j in range(3):
        print(backTraceList[solutionPos[0]-i][0][j])
    
    print("----")
    
    
for i in range(1, solutionPos[0]+1):
    print(backTraceList[solutionPos[0]-i][ORIGIN_POS])

    
nodePath = open("nodePath.txt", "w+")
for i in range(len(backTraceList)):
    writeNodes(backTraceList[i], nodePath)
nodePath.close()

nodes = open("nodes.txt", "w+")
for i in range(len(stateList)):
    for j in range(len(stateList[i])):
        writeNodes(stateList[i][j], nodes)
nodes.close()


nodeInfo = open("nodeInfo.txt", "w+")
nodeInfo.write("Node Level, Node Iteration,      Parent Level, Parent Iteration\n")
for i in range(len(backTraceList)):
    writeInfo(backTraceList[len(backTraceList)-1-i], nodeInfo)
nodeInfo.close()

[1, 2, 3]
[4, 5, 6]
[7, 0, 8]
----
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
----
Right
